In [22]:
import boto3
from pprint import pprint
import csv
import json
import random

ddb_table = 'awseb-e-7c68nzyeq7-stack-MCIDIdeasTable-PH0OQLXZC3RI'


In [ ]:
# Get access credentials for Dynamo


ssmclient = boto3.client('ssm')
ssmresponse = ssmclient.get_parameter(
    Name='DynamoDB_Reader_User_Creds',
    WithDecryption=True
)

ddb_creds = json.loads(ssmresponse['Parameter']['Value'])
ddb_access_key = ddb_creds['Access key ID']
ddb_secret_key = ddb_creds['Secret access key']

In [24]:
# Scan the Dynamo table

ddbclient = boto3.client(
    'dynamodb',
    aws_access_key_id=ddb_access_key,
    aws_secret_access_key=ddb_secret_key,
    region_name='us-east-1'
    )

ddbresponse = ddbclient.scan(TableName=ddb_table)



In [ ]:
# Write output from table to CSV

if ddbresponse['Count'] > 0:

    table_header=list(ddbresponse['Items'][0].keys())

    # open a file for writing
    data = open('ddb_output.csv', 'w')

    # create the csv writer object
    csvwriter = csv.writer(data)
    count = 0
    for item in response['Items']:
        if count == 0:
            header = table_header
            csvwriter.writerow(header)
            count += 1
        row = [item[attrib]['S'] for attrib in table_header]
        csvwriter.writerow(row)
    data.close()

    print('Success! DDB table written to ddb_output.csv')

else:
    print('Dynamo table is empty. No output written to disk.')


In [25]:
# pprint(ddbresponse)

{'Count': 5,
 'Items': [{'Company': {'S': 'djfkljsdkl '},
            'DateTime': {'S': 'Sat, 02 Mar 2019 16:53:18 GMT'},
            'Email': {'S': 'dskfjsdklj'},
            'Idea': {'S': 'dkfjl akjf'},
            'Name': {'S': 'Jimdklff adsklfj s'},
            'SessionID': {'S': ' - Houston - January 1, 2019'},
            'UUID': {'S': '7a55439d-0f52-4d75-90c6-863b9d19251a'}},
           {'Company': {'S': 'dfkdjf dlkf '},
            'DateTime': {'S': 'Sat, 02 Mar 2019 16:53:00 GMT'},
            'Email': {'S': 'dfjkslf 1k3'},
            'Idea': {'S': 'j'},
            'Name': {'S': 'Jim'},
            'SessionID': {'S': ' - Houston - January 1, 2019'},
            'UUID': {'S': '664eb21c-467d-4e21-8eed-7378ebe3a1b4'}},
           {'Company': {'S': 'Hello Yo'},
            'DateTime': {'S': 'Sat, 02 Mar 2019 16:30:59 GMT'},
            'Email': {'S': 'Wallace@something.com'},
            'Idea': {'S': 'something aawesome'},
            'Name': {'S': 'Wallace'},
            'Sess

In [71]:
# Select random winner

winner = random.choice(range(ddbresponse['Count']))

print('Winner is Idea #' + str(winner))
print(ddbresponse['Items'][winner]['Name']['S'])

Winner is Idea #2
Wallace


4
3
0
0
3
3
1
2
3
0
0
0
2
1
3
2
4
2
1
0
1
3
1
1
0
3
2
4
0
3
0
1
3
4
1
0
2
4
0
3
3
3
3
0
1
1
1
4
4
4
1
4
3
3
2
4
0
3
0
4
3
1
3
3
0
3
4
3
1
2
0
4
0
0
2
0
0
0
0
3
3
4
3
0
0
2
0
1
0
4
2
3
4
3
4
1
2
0
4
3
